## Cross Database Joins

In [6]:
dataset_id = "reporting"

In [7]:
!bq --location=US mk --dataset {dataset_id}

Dataset 'starry-center-266501:reporting' successfully created.


Look at movies that have the same title and release date in imdb_modeled and bollywood modeled and compare the director names

In [3]:
%%bigquery
SELECT t.title, d.director AS bollyDirector, primaryName as imdbDirector, EXTRACT(YEAR FROM releaseDate) AS year
FROM bollywood_modeled.bollywoodTitles_Beam_DF t
LEFT JOIN bollywood_modeled.bollywoodDirects_Beam_DF d
ON t.title = d.title
JOIN imdb_modeled.title_Beam_DF tb
ON t.title = tb.primaryTitle
AND EXTRACT(YEAR FROM releaseDate) = tb.startYear
JOIN imdb_modeled.Directs_Beam_DF db 
ON tb.tConst = db.tConst
JOIN imdb_modeled.People p
ON p.nConst = db.director

,title,bollyDirector,imdbDirector,year
0,Lal Patthar,Sushil Majumdar,Sushil Majumdar,1971
1,Aurangzeb,Atul Sabharwal,Atul Sabharwal,2013
2,Shiv Parvati,S. N. Tripathi,S.N. Tripathi,1962
3,Dev Kanya,S. N. Tripathi,S.N. Tripathi,1963
4,Dev Kanya,Dhirubhai Desai,S.N. Tripathi,1963
...,...,...,...,...
7559,Baghi,Ram Dayal,Anant Thakur,1953
7560,Paras,Anant Thakur,Anant Thakur,1949
7561,Paras,C.P. Dixit,Anant Thakur,1949
7562,Ek Naya Itihas,Unknown,B.S. Narayan,1984


## Create a view that is the count of all of the movies that bollywood released by year that are in both the bollywood database and the imdb database

In [8]:
%%bigquery
CREATE OR REPLACE VIEW reporting.v_sharedMoviesByYear AS
SELECT year, COUNT (year) AS moviesReleased
FROM
(SELECT t.title, d.director AS bollyDirector, primaryName as imdbDirector, EXTRACT(YEAR FROM releaseDate) AS year
FROM `starry-center-266501.bollywood_modeled.bollywoodTitles_Beam_DF` t
LEFT JOIN `starry-center-266501.bollywood_modeled.bollywoodDirects_Beam_DF` d
ON t.title = d.title
JOIN `starry-center-266501.imdb_modeled.title_Beam_DF` tb
ON t.title = tb.primaryTitle
AND EXTRACT(YEAR FROM releaseDate) = tb.startYear
JOIN `starry-center-266501.imdb_modeled.Directs_Beam_DF` db 
ON tb.tConst = db.tConst
JOIN `starry-center-266501.imdb_modeled.People` p
ON p.nConst = db.director)
GROUP BY year
ORDER BY year ASC

""


## Look at genres in the bollywood table versus genres in the imdb genres table for titles that are in both data sets

In [9]:
%%bigquery
SELECT bt.title, bg.genre AS bollyGenre, genres AS imdbGenre
FROM bollywood_modeled.bollywoodGenres_Beam_DF bg
JOIN bollywood_modeled.bollywoodTitles_Beam_DF bt
ON bg.title = bt.title
JOIN imdb_modeled.title_Beam_DF 
ON bt.title = primaryTitle
AND EXTRACT(year FROM releaseDate) = startYear

,title,bollyGenre,imdbGenre
0,Jurm,thriller,Crime
1,Bhule Bhatke,action,Comedy
2,Dayavan,crime,Action
3,Don Muthu Swami,comedy,Comedy
4,Phir Bhi Apna Hai,social,Fantasy
...,...,...,...
6226,Aaya Toofan,action,Action
6227,Bhavna,drama,Drama
6228,Hamari Beti,family,Drama
6229,Rock On!!,musical,Drama


In [10]:
%%bigquery
CREATE OR REPLACE VIEW reporting.v_sharedMoviesGenres AS
SELECT bt.title, bg.genre AS bollyGenre, genres AS imdbGenre
FROM `starry-center-266501.bollywood_modeled.bollywoodGenres_Beam_DF` bg
JOIN `starry-center-266501.bollywood_modeled.bollywoodTitles_Beam_DF` bt
ON bg.title = bt.title
JOIN `starry-center-266501.imdb_modeled.title_Beam_DF`
ON bt.title = primaryTitle
AND EXTRACT(year FROM releaseDate) = startYear

""


### Look at the croresGrossed reported by the bollywood db versus the average rating reported by the imdb db

In [11]:
%%bigquery
SELECT DISTINCT bt.title, croresGrossed, averageRating
FROM bollywood_modeled.bollywoodTitles_Beam_DF bt
JOIN imdb_modeled.title_Beam_DF t
ON primaryTitle = bt.title
AND startYear = EXTRACT(YEAR FROM releaseDate)
JOIN imdb_modeled.TitleRatings tr
ON tr.tConst = t.tConst
WHERE croresGrossed > 0
ORDER BY averageRating

,title,croresGrossed,averageRating
0,Bodyguard,234.390000000,4.6
1,Dhoom 3,589.200000000,5.4
2,Ek Tha Tiger,334.390000000,5.5
3,Tiger Zinda Hai,565.100000000,6.0
4,Hum Saath-Saath Hain: We Stand United,81.710000000,6.2
5,Dabangg,250.000000000,6.2
6,Raja Hindustani,87.500000000,6.2
7,Beta,23.500000000,6.3
8,Dhoom 2,151.390000000,6.5
9,No Entry,97.640000000,6.6


In [12]:
%%bigquery
CREATE OR REPLACE VIEW reporting.v_Crores_vs_Rating AS
SELECT DISTINCT bt.title, croresGrossed, averageRating
FROM `starry-center-266501.bollywood_modeled.bollywoodTitles_Beam_DF` bt
JOIN `starry-center-266501.imdb_modeled.title_Beam_DF` t
ON primaryTitle = bt.title
AND startYear = EXTRACT(YEAR FROM releaseDate)
JOIN `starry-center-266501.imdb_modeled.TitleRatings` tr
ON tr.tConst = t.tConst
WHERE croresGrossed > 0
ORDER BY averageRating

""
